### mysql과 python연동
1. 라이브러리 설치 
2. sql 서버 정보를 등록 & 연결
3. sql 쿼리문을 이용하여 데이터 CRUD 작업

In [ ]:
# 라이브러리 설치 
!pip install pymysql

In [2]:
import pymysql 

In [3]:
import pandas as pd

In [4]:
# mysql 서버의 정보를 등록
_db = pymysql.connect(
    host = 'localhost', 
    user = 'root', 
    password = '1234', # 패스워드 설정이 되어있는 분들만 작성
    db = 'ubion8', 
    port = 3306
)

In [5]:
# cursor 생성
# sql 쿼리문을 실행시키기 위한 필수 변수 
# sql 쿼리문을 실행 한 결과를 dict형태로 받기위해 설정 
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [10]:
sql = "select * from user"

# sql 서버에 쿼리문을 실행
cursor.execute(sql)

4

In [11]:
cursor.fetchall()

[{'id': 'test', 'password': '1234'},
 {'id': 'test2', 'password': '1111'},
 {'id': 'test3', 'password': '0000'},
 {'id': 'test9', 'password': '1111'}]

In [8]:
sql = """
    insert into 
    user 
    values 
    ('test9', '1111')
"""

In [9]:
cursor.execute(sql)

1

In [12]:
## DB에 데이터의 변화가 존재하면 mysql server commit
_db.commit()

### sql 쿼리문을 실행하는 함수 생성
1. 첫번째 함수 
    - 매개변수 1개 (sql 쿼리문)
    - sql 쿼리문을 실행하고 결과를 데이터프레임으로 리턴
2. 두번째 함수 
    - 매개변수 1개 (sql 쿼리문)
    - sql 쿼리문을 실행하고 데이터베이스를 변경
3. 세번째 함수 
    - 1번 함수와 2번 함수를 결합

In [26]:
def sql_select(sql, value = []):
    # 커서를 이용해서 sql 쿼리문을 실행
    cursor.execute(sql, value)
    # 실행 된 쿼리에서 결과값을 변수에 대입
    result = cursor.fetchall()
    result = pd.DataFrame(result)
    return result

In [27]:
sql = """
    select 
    * 
    from 
    user
"""
sql_select(sql)

,id,password
0,test,1111
1,test2,1111
2,test3,0000
3,test9,1111


In [28]:
sql = """
    select 
    * 
    from 
    user 
    where 
    id = %s
"""
value = ['test']
sql_select(sql, value)

,id,password
0,test,1111


In [29]:
def sql_cud(sql, value = []):
    # 커서를 사용하여 sql 쿼리문을 실행
    cursor.execute(sql, value)
    # 데이터베이스 변경
    _db.commit()
    return "Query OK"

In [17]:
sql = """
    update 
    user 
    set 
    password = '1111'
    where 
    id = 'test'
"""
sql_cud(sql)

'Query OK'

In [22]:
sql = """
    select 
    * 
    from 
    user 
"""
value = []

cursor.execute(sql, value)
cursor.fetchall()

[{'id': 'test', 'password': '1111'},
 {'id': 'test2', 'password': '1111'},
 {'id': 'test3', 'password': '0000'},
 {'id': 'test9', 'password': '1111'}]

In [30]:
sql = """
    insert into 
    user 
    values 
    (%s, %s)
"""
value = ['test8', '1234']
sql_cud(sql, value)

'Query OK'

In [31]:
sql = """
    select 
    * 
    from 
    user
"""

sql

'\n    select \n    * \n    from \n    user\n'

### 문제 
1. sql_select(sql, value=[])함수와 sql_cud(sql, value=[])함수를 결합
    - sql 매개변수에 들어오는 인자값이 select문이면 sql_select에서 작성한 코드가 실행
    - select문이 아니라면 sql_cud에서 작성한 코드가 실행

In [35]:
sql

'select * from user'

In [47]:
sql = "SELECT * from user"
sql2 = """
    select 
    * 
    from 
    user
"""
sql.lower().lstrip().startswith('select')

True

In [48]:
def query(sql, value = []):
    cursor.execute(sql, value)
    # 쿼리문이 select문이라면
    if sql.lstrip().lower().startswith('select'):
        result = cursor.fetchall()
        result = pd.DataFrame(result)
    else:
        _db.commit()
        result = "Query OK"
    return result
    

In [49]:
sql = """
    delete 
    from 
    user 
    where 
    id = %s
"""
input_id = input("아이디를 입력하시오")
value = [input_id]

query(sql, value)

'Query OK'

In [50]:
sql = "select * from user"
query(sql)

,id,password
0,test,1111
1,test2,1111
2,test3,0000
3,test8,1234


### class로 생성
1. 변수 (```__init__ 함수를 이용하여 변수 생성```)
    - _db : 서버의 정보를 입력하고 접속
    - cursor : DB에 sql쿼리문을 실행시키기 위한 변수
2. 함수
    - query() : sql 쿼리문을 인자값으로 받아서 sql쿼리문을 실행

In [51]:
class Database:
    # 생성자 함수 
    def __init__(self, _host, _user, _pass, _db, _port):
        self.db = pymysql.connect(
            host = _host, 
            user = _user, 
            password = _pass, 
            db = _db, 
            port = _port
        )
        self.cursor = self.db.cursor(pymysql.cursors.DictCursor)

    def query(self, sql, value = []):
        # 커서를 이용해서 sql 쿼리문 실행
        self.cursor.execute(sql, value)

        # sql 쿼리문의 시작이 select 라면
        if sql.lstrip().lower().startswith('select'):
            result = self.cursor.fetchall()
            result = pd.DataFrame(result)
        else:
            self.db.commit()
            result = 'Query Ok'
        return result

In [53]:
test_db = Database(
    "172.16.12.156", 
    "ubion", 
    "1234", 
    "ubion8",
    3306
)

In [54]:
sql = """
    select 
    * 
    from 
    user
"""

test_db.query(sql)

,id,password
0,test,1111
1,test2,1111
2,test3,0000
3,test8,1234


In [56]:
test_db2 = Database(
    'darkpreist.iptime.org', 
    'ubion', 
    '1234', 
    'ubion', 
    3306
)

In [57]:
sql = """
    select 
    * 
    from 
    tran_1
"""
test_db2.query(sql)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019
2,T0000000115,120000,2019-02-01 02:34:19,HD298120
3,T0000000116,210000,2019-02-01 02:47:23,IK452215
4,T0000000117,170000,2019-02-01 04:33:46,PL542865
...,...,...,...,...
4995,T0000005108,210000,2019-06-15 02:42:41,HD315748
4996,T0000005109,150000,2019-06-15 03:36:16,HI215420
4997,T0000005110,50000,2019-06-15 03:44:06,IK880102
4998,T0000005111,210000,2019-06-15 04:14:06,IK074758


In [58]:
import sql_mod

In [61]:
# sql_mod 모듈에서 Database 클래스를 생성
mod_db = sql_mod.Database(
    'darkpreist.iptime.org', 
    'ubion', 
    '1234', 
    'ubion', 
    3306
)

OperationalError: (2003, "Can't connect to MySQL server on 'darkpreist.iptime.org' (timed out)")

In [60]:
sql = """
    select 
    * 
    from 
    emp
"""
mod_db.query(sql)

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902,1980-12-17,800.00,None,20
1,7499,ALLEN,SALESMAN,7698,1981-02-20,1600.00,300.00,30
2,7521,WARD,SALESMAN,7698,1981-02-22,1250.00,500.00,30
3,7566,JONES,MANAGER,7839,1981-04-02,2975.00,None,20
4,7654,MARTIN,SALESMAN,7698,1981-09-28,1250.00,1400.00,30
5,7698,BLAKE,MANAGER,7839,1981-05-01,2850.00,None,30
6,7782,CLARK,MANAGER,7839,1981-06-09,2450.00,None,10
7,7788,SCOTT,ANALYST,7566,1987-06-28,3000.00,None,20
8,7839,KING,PRESIDENT,None,1981-11-17,5000.00,None,10
9,7844,TURNER,SALESMAN,7698,1981-09-08,1500.00,0.00,30
